In [1]:
import stim
import numpy as np
from numpy.linalg import matrix_power, matrix_rank
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
import os
from tqdm import tqdm
import galois
import qldpc
from qldpc.codes.classical import ClassicalCode, HammingCode
from qldpc.codes.quantum import HGPCode

In [2]:
def get_classical_code(r):
    '''
    Generate a classical Hamming code with r parity bits.
    Hamming codes are a family of linear error-correcting codes that can detect and correct single-bit errors.
    They are defined by their parameters (n, k), where n is the length of the codeword and k is the number of information bits.
    ''' 
    code = HammingCode(r)
    return code.matrix

In [ ]:
def generate_HGP_code(ka, kb):
    """
    Generate a HGP code from two classical codes.
    ka: the number of information bits in code_a
    kb: the number of information bits in code_b
    The HGP code is a quantum error-correcting code that is constructed from two linear classical codes.   
    """
    code_a = get_classical_code(ka)
    code_b = get_classical_code(kb)
    HGP_code = HGPCode(code_a, code_b)
    # matrix_x = HGP_code.matrix_x
    # matrix_z = HGP_code.matrix_z
    # print(f"parity_matrix_x.shape: {matrix_x.shape}")
    # print(f"parity_matrix_z.shape: {matrix_z.shape}")
    # print(f"parity_matrix_x: {matrix_x}")
    # print(f"parity_matrix_z: {matrix_z}")
    return HGP_code

In [30]:
from utils import par2gen, get_logicals, SGSOP

def add_observable(logicalX, logicalZ = None):
    circuit = stim.Circuit()
    for i, logical in enumerate(logicalX):
        include_qubits = np.where(logical == 1)[0]
        include_qubits = [-j-1 for j in include_qubits]
        circuit.append_operation("OBSERVABLE_INCLUDE", [stim.target_rec(q) for q in include_qubits])
    # for i, logical in enumerate(logicalZ):
    #     include_qubits = np.where(logical == 1)[0]
    #     include_qubits = [-j-1 for j in include_qubits]
    #     circuit.append_operation("OBSERVABLE_INCLUDE", [stim.target_rec(q) for q in include_qubits])
    return circuit

def generate_circuit(HGP_Code):
    """
    Generate a stim circuit from the stabilizer generators of a quantum code.
    matrix_x: the stabilizer generators for X-type errors
    matrix_z: the stabilizer generators for Z-type errors
    The circuit is generated by appending operations for each stabilizer generator.
    """
    Hx, Hz = HGP_Code.matrix_x, HGP_Code.matrix_z
    Hx = np.array(Hx).astype(int)
    Hz = np.array(Hz).astype(int)
    commutator = np.dot(Hx, Hz.T) % 2  # 计算对易关系
    if np.any(commutator):
        print("Error: Stabilizer generators are not orthogonal!")
    else:
        print("Stabilizer generators are orthogonal.")
    # print(f"parity_matrix_x: {Hx}")
    # print(f"parity_matrix_z: {Hz}")
    n = Hx.shape[1]
    mx, mz = Hx.shape[0], Hz.shape[0]
    m = mx + mz
    # print(f"n: {n}, m: {m}")
    # print(f"parity_matrix_x.shape: {Hx.shape}")
    # print(f"parity_matrix_z.shape: {Hz.shape}")
    Gx, col_Gx = par2gen(Hx)
    Gz, col_Gz = par2gen(Hz)
    logicals, generators = SGSOP(Gx, Gz, n)
    logX = np.array([l[1][n:] for l in logicals])
    logZ = np.array([l[0][:n] for l in logicals])
    circuit = stim.Circuit()    
    for log in logX:
        if np.any(np.dot(log, Hz.T) % 2):
            print("Error: Logical X anti-commutes with Z stabilizers!")
    for log in logZ:
        if np.any(np.dot(log, Hx.T) % 2):
            print("Error: Logical Z anti-commutes with X stabilizers!")
    # circuit.append("R", [i for i in range(n)])
    print(circuit)
    for stabilizer in Hx:
        detector = []
        for i, qubit in enumerate(stabilizer):
            if qubit == 1:
                detector.append(stim.target_x(i))
                detector.append(stim.target_combiner())
        if detector != None:
            detector.pop()
        circuit.append_operation("MPP", detector)
    for stabilizer in Hz:
        detector = []
        for i, qubit in enumerate(stabilizer):
            if qubit == 1:
                detector.append(stim.target_z(i))
                detector.append(stim.target_combiner())
        if detector != None:
            detector.pop()
        circuit.append_operation("MPP", detector)
    circuit.append("DEPOLARIZE1", [i for i in range(n)], 0)
    # circuit.append("X_ERROR", [0, 1, 2], 0.1)

    for stabilizer in Hx:
        detector = []
        for i, qubit in enumerate(stabilizer):
            if qubit == 1:
                detector.append(stim.target_x(i))
                detector.append(stim.target_combiner())
        if detector != None:
            detector.pop()
        circuit.append_operation("MPP", detector)
    for stabilizer in Hz:
        detector = []
        for i, qubit in enumerate(stabilizer):
            if qubit == 1:
                detector.append(stim.target_z(i))
                detector.append(stim.target_combiner())
        if detector != None:
            detector.pop()
        circuit.append_operation("MPP", detector)

    for i in range(1, m + 1):
        circuit.append("DETECTOR", [stim.target_rec(-i), stim.target_rec(-i-m)])
    # circuit.append_operation("H", [i for i in range(n)])
    circuit.append_operation("M", [i for i in range(n)][::-1])
    circuit += add_observable(logZ)
    # Add a measurement operation for the logical qubits
    return circuit
HGP_Code = generate_HGP_code(3, 3)
circuit = generate_circuit(HGP_Code)
# circuit.diagram('detslice-with-ops-svg', tick=range(0, 5), filter_coords=['L0'])
# print(circuit)

Stabilizer generators are orthogonal.



In [31]:
from bp_osd import BPOSD

sampler = circuit.compile_detector_sampler()
dem = circuit.detector_error_model()
decoder = BPOSD(dem, max_bp_iters = 20) #使用BP_OSD
# print(f"dem: {dem}")
dem.diagram("matchgraph-svg")
# # sample 10 shots
N = 50
syndrome, actual_observables = sampler.sample(shots=N, separate_observables=True)
# print("Syndrome shape:", syndrome.shape)
# print("First few syndromes:", syndrome[:5])

predicted_observables = decoder.decode_batch(syndrome)

# print(f"actual_observable:{actual_observables}\npredicted_observables:{predicted_observables}")

num_errors = np.sum(np.any(predicted_observables != actual_observables, axis=1))
print(f'Error rate: {num_errors / N * 100}%')

Check matrix shape: (42, 0)
Error rate: 0.0%
